<a href="https://colab.research.google.com/github/Eman-Ehab/DLND/blob/master/timesformer_feature_extractions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
pip install -q pytorchvideo transformers evaluate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.7/132.7 kB 2.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 46.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 kB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.2/50.2 kB 5.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.0/31.0 MB 38.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 4.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 26.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 99.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 73.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 492.2/492.2 kB 41.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 12.7 MB/s eta 0:0

In [3]:
import av
import torch
import numpy as np
from torch import nn
from transformers import TimesformerConfig, TimesformerModel
from transformers import AutoImageProcessor, TimesformerForVideoClassification
from huggingface_hub import hf_hub_download

np.random.seed(0)

In [ ]:
videos_path = "./drive/MyDrive/FeatureExtraction/video_input/"
video_list = os.listdir(videos_path)



In [4]:
pretrained_timesformer = ["facebook/timesformer-base-finetuned-k400",
                        "facebook/timesformer-base-finetuned-k600",
                        "facebook/timesformer-hr-finetuned-k600",
                        "facebook/timesformer-hr-finetuned-k400",
                        "facebook/timesformer-base-finetuned-ssv2", "facebook/timesformer-hr-finetuned-ssv2"]


In [27]:
model_names = [
    # Kinetics-400 checkpoints (hr = high resolution input of 448px instead of 224px)
    "timesformer-base-finetuned-k400",
    "timesformer-large-finetuned-k400",
    "timesformer-hr-finetuned-k400",
    # Kinetics-600 checkpoints (hr = high resolution input of 448px instead of 224px)
    "timesformer-base-finetuned-k600",
    "timesformer-large-finetuned-k600",
    "timesformer-hr-finetuned-k600",
    # Something-Something-v2 checkpoints (hr = high resolution input of 448px instead of 224px)
    "timesformer-base-finetuned-ssv2",
    "timesformer-large-finetuned-ssv2",
    "timesformer-hr-finetuned-ssv2",
]
model_names = ["facebook/"+md for md in model_names]

In [ ]:
image_processor = AutoImageProcessor.from_pretrained("facebook/timesformer-base-finetuned-k400")
configuration.update_from_string("num_hidden_layers=4")

model = TimesformerModel.from_pretrained("facebook/timesformer-base-finetuned-k400", config=
                                         configuration)

In [ ]:
model = TimesformerForVideoClassification.from_pretrained(pre_model)
configuration = TimesformerConfig()

In [55]:
#videos_path = "./drive/MyDrive/FeatureExtraction/video_input/"
#video_list = os.listdir(videos_path)

video_list = ["eating_spaghetti.mp4"]


for pre_model in model_names[:1]:
  print(pre_model)
  confgiuration = TimesformerConfig()

  model = TimesformerForVideoClassification.from_pretrained(pre_model)
  config_dict = model.config.to_dict()


  print(model.config.num_hidden_layers+1)
  print(">>>>>>>>>>>>>>>>>>>>>>>>>>")
  # model layers
  for i in range(1, model.config.num_hidden_layers+1):
    # --------------------

    # adjust config and update
    config_dict["num_hidden_layers"] = i

    configuration.update(config_dict=config_dict)

    model_layers = TimesformerForVideoClassification.from_pretrained(pre_model, config=configuration)
    model_layers.eval()

    # model layers
    # ----------------

    image_processor = AutoImageProcessor.from_pretrained(pre_model)
    #model_layers.eval()

    for file_path in video_list:
      file_path = hf_hub_download(repo_id="nielsr/video-demo", filename=file_path, repo_type="dataset")

      container = av.open(file_path)
      # sample 8 frames
      indices = sample_frame_indices(clip_len=8, frame_sample_rate=1, seg_len=container.streams.video[0].frames)

      video = read_video_pyav(container, indices)

      inputs = image_processor(list(video), return_tensors="pt")

      with torch.no_grad():

        # Features is the output
        outputs = model_layers(**inputs)
        logits = outputs.logits

      predicted_label = logits.argmax(-1).item()
      print(f"Num of model layers: {i}")
      print(model.config.id2label[predicted_label])
      print("-----------------------------------------------------------------------------------------------------")
      #break





facebook/timesformer-base-finetuned-k400
13
>>>>>>>>>>>>>>>>>>>>>>>>>>


Some weights of the model checkpoint at facebook/timesformer-base-finetuned-k400 were not used when initializing TimesformerForVideoClassification: ['timesformer.encoder.layer.9.intermediate.dense.bias', 'timesformer.encoder.layer.1.attention.output.dense.weight', 'timesformer.encoder.layer.5.output.dense.weight', 'timesformer.encoder.layer.8.layernorm_after.bias', 'timesformer.encoder.layer.10.attention.output.dense.weight', 'timesformer.encoder.layer.10.temporal_attention.output.dense.bias', 'timesformer.encoder.layer.4.temporal_dense.weight', 'timesformer.encoder.layer.2.output.dense.bias', 'timesformer.encoder.layer.5.temporal_dense.weight', 'timesformer.encoder.layer.8.temporal_attention.output.dense.bias', 'timesformer.encoder.layer.11.attention.output.dense.weight', 'timesformer.encoder.layer.4.temporal_attention.output.dense.bias', 'timesformer.encoder.layer.5.temporal_dense.bias', 'timesformer.encoder.layer.4.layernorm_before.bias', 'timesformer.encoder.layer.4.attention.atten

Num of model layers: 1
throwing ball
-----------------------------------------------------------------------------------------------------


Some weights of the model checkpoint at facebook/timesformer-base-finetuned-k400 were not used when initializing TimesformerForVideoClassification: ['timesformer.encoder.layer.9.intermediate.dense.bias', 'timesformer.encoder.layer.5.output.dense.weight', 'timesformer.encoder.layer.8.layernorm_after.bias', 'timesformer.encoder.layer.10.attention.output.dense.weight', 'timesformer.encoder.layer.10.temporal_attention.output.dense.bias', 'timesformer.encoder.layer.4.temporal_dense.weight', 'timesformer.encoder.layer.2.output.dense.bias', 'timesformer.encoder.layer.5.temporal_dense.weight', 'timesformer.encoder.layer.8.temporal_attention.output.dense.bias', 'timesformer.encoder.layer.11.attention.output.dense.weight', 'timesformer.encoder.layer.4.temporal_attention.output.dense.bias', 'timesformer.encoder.layer.5.temporal_dense.bias', 'timesformer.encoder.layer.4.layernorm_before.bias', 'timesformer.encoder.layer.4.attention.attention.qkv.weight', 'timesformer.encoder.layer.6.output.dense.w

Num of model layers: 2
shaking hands
-----------------------------------------------------------------------------------------------------


...

Num of model layers: 9
peeling potatoes
-----------------------------------------------------------------------------------------------------


Some weights of the model checkpoint at facebook/timesformer-base-finetuned-k400 were not used when initializing TimesformerForVideoClassification: ['timesformer.encoder.layer.10.attention.output.dense.weight', 'timesformer.encoder.layer.10.temporal_attention.output.dense.bias', 'timesformer.encoder.layer.11.temporal_attention.attention.qkv.bias', 'timesformer.encoder.layer.10.temporal_dense.weight', 'timesformer.encoder.layer.11.attention.output.dense.weight', 'timesformer.encoder.layer.10.temporal_dense.bias', 'timesformer.encoder.layer.11.temporal_layernorm.bias', 'timesformer.encoder.layer.11.output.dense.weight', 'timesformer.encoder.layer.11.temporal_attention.output.dense.weight', 'timesformer.encoder.layer.10.temporal_attention.output.dense.weight', 'timesformer.encoder.layer.10.layernorm_after.bias', 'timesformer.encoder.layer.11.temporal_layernorm.weight', 'timesformer.encoder.layer.11.layernorm_after.bias', 'timesformer.encoder.layer.11.attention.output.dense.bias', 'timesfo

Num of model layers: 10
shaking head
-----------------------------------------------------------------------------------------------------


Some weights of the model checkpoint at facebook/timesformer-base-finetuned-k400 were not used when initializing TimesformerForVideoClassification: ['timesformer.encoder.layer.11.attention.attention.qkv.bias', 'timesformer.encoder.layer.11.temporal_attention.attention.qkv.bias', 'timesformer.encoder.layer.11.layernorm_before.bias', 'timesformer.encoder.layer.11.attention.output.dense.weight', 'timesformer.encoder.layer.11.temporal_attention.attention.qkv.weight', 'timesformer.encoder.layer.11.intermediate.dense.bias', 'timesformer.encoder.layer.11.temporal_layernorm.bias', 'timesformer.encoder.layer.11.output.dense.weight', 'timesformer.encoder.layer.11.intermediate.dense.weight', 'timesformer.encoder.layer.11.temporal_attention.output.dense.bias', 'timesformer.encoder.layer.11.temporal_attention.output.dense.weight', 'timesformer.encoder.layer.11.temporal_layernorm.weight', 'timesformer.encoder.layer.11.layernorm_after.bias', 'timesformer.encoder.layer.11.layernorm_after.weight', 'tim

Num of model layers: 11
eating spaghetti
-----------------------------------------------------------------------------------------------------
Num of model layers: 12
eating spaghetti
-----------------------------------------------------------------------------------------------------


In [19]:
image_processor = AutoImageProcessor.from_pretrained(model_names[0], config=configuration)

In [ ]:
configuration

TimesformerConfig {
  "_name_or_path": "facebook/timesformer-base-finetuned-k400",
  "attention_probs_dropout_prob": 0.0,
  "attention_type": "divided_space_time",
  "drop_path_rate": 0,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.0,
  "hidden_size": 768,
  "image_size": 224,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-06,
  "model_type": "timesformer",
  "num_attention_heads": 12,
  "num_channels": 3,
  "num_frames": 8,
  "num_hidden_layers": 10,
  "patch_size": 16,
  "qkv_bias": true,
  "transformers_version": "4.31.0"
}

In [29]:
def read_video_pyav(container, indices):
    '''
    Decode the video with PyAV decoder.
    Args:
        container (`av.container.input.InputContainer`): PyAV container.
        indices (`List[int]`): List of frame indices to decode.
    Returns:
        result (np.ndarray): np array of decoded frames of shape (num_frames, height, width, 3).
    '''
    frames = []
    container.seek(0)
    start_index = indices[0]
    end_index = indices[-1]
    for i, frame in enumerate(container.decode(video=0)):
        if i > end_index:
            break
        if i >= start_index and i in indices:
            frames.append(frame)
    return np.stack([x.to_ndarray(format="rgb24") for x in frames])


def sample_frame_indices(clip_len, frame_sample_rate, seg_len):
    converted_len = int(clip_len * frame_sample_rate)
    end_idx = np.random.randint(converted_len, seg_len)
    start_idx = end_idx - converted_len
    indices = np.linspace(start_idx, end_idx, num=clip_len)
    indices = np.clip(indices, start_idx, end_idx - 1).astype(np.int64)
    return indices


In [54]:
model = TimesformerForVideoClassification.from_pretrained("facebook/timesformer-base-finetuned-k400")

inputs = image_processor(list(video), return_tensors="pt")

with torch.no_grad():

    outputs = model(**inputs)

    logits = outputs.logits

# model predicts one of the 400 Kinetics-400 classes

predicted_label = logits.argmax(-1).item()

print(model.config.id2label[predicted_label])

crying
